In [1]:
import matplotlib
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style, colors
from matplotlib import gridspec

matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (8, 6)

freeze list 123 patients: HTMCP-03-06-02061, squamous, hiv -, PIK3CA mutated' should exclude this sample
removed two hyper mutated cases:['HTMCP-03-06-02007', 'HTMCP-03-06-02026', 'HTMCP-03-06-02061']

In [53]:
# snv summary file
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/HIGH_MODERATE_SNV_summary_with_normal_with_AF_filtered_somatic.tsv'
df1 = pd.read_csv(f, sep='\t', usecols=['gene', 'patient_ID'])
df1.head(2)

,gene,patient_ID
0,SAMD11,HTMCP-03-06-02007_malignant
1,SAMD11,HTMCP-03-06-02214_malignant


In [54]:
# indel summary file
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/HIGH_MODERATE_INDEL_summary_with_normal_strelka_indels_only.tsv'
df2 = pd.read_csv(f, sep='\t', usecols=['gene', 'patient_ID'])
df2.head(2)

,gene,patient_ID
0,PCNT,HTMCP-03-06-02448_malignant
1,MLL3,HTMCP-03-06-02097_malignant


In [55]:
dfc = pd.concat([df1, df2])
dfc['patient'] = dfc.patient_ID.str.replace('_malignant', '')
dfc.drop('patient_ID', inplace=True, axis=1)

In [56]:
df1.shape

(44053, 2)

In [57]:
df2.shape 

(827, 2)

In [58]:
dfc.shape

(44880, 2)

In [59]:
dfc.head(2)

,gene,patient
0,SAMD11,HTMCP-03-06-02007
1,SAMD11,HTMCP-03-06-02214


remove hyper_mutated cases

In [60]:
dfc = dfc[~dfc['patient'].isin(['HTMCP-03-06-02007', 'HTMCP-03-06-02026', 'HTMCP-03-06-02061'])]

In [61]:
dfd = dfc.drop_duplicates()

In [62]:
dfe = dfd.groupby('gene').count()

In [63]:
dfe.head(2)

,patient
gene,
A1BG,1
A1CF,2


In [51]:
mutated_genes = dfe.index.tolist()

In [52]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/all_mutated_genes.txt'
with open(f, 'w') as fh:
    for gene in mutated_genes:
        fh.write('{}\n'.format(gene))

In [64]:
dff = dfe[dfe.patient >= 3].sort_values('patient')

In [65]:
dff.head()

,patient
gene,
hsa-mir-4763,3
HSPA6,3
HSP90AA1,3
HS6ST1,3
HS3ST5,3


In [66]:
dff.shape

(2462, 1)

In [34]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/validation_gene_list.txt'
dff.to_csv(of)

In [58]:
of2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients/tcga_genes_mutated_in_3_or_mroe_patients.txt'
with open(of2, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in dfc:
        writer.writerow([val]) 